In [1]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install transformers diffusers datasets accelerate


Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoModel
from diffusers import DiffusionPipeline
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from datasets import load_dataset
import matplotlib.pyplot as plt
from diffusers import AutoPipelineForText2Image

C:\Users\tomma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
discriminator = AutoModel.from_pretrained("facebook/dinov2-base")
teacher_model = AutoPipelineForText2Image.from_pretrained("warp-ai/wuerstchen", torch_dtype=torch.float16)

student_model = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1")
# Access the UNet model within the pipeline for optimization
unet_model = student_model.unet

README.md: 100%|██████████| 5.28k/5.28k [00:00<?, ?B/s]s]
tokenizer/special_tokens_map.json: 100%|██████████| 472/472 [00:00<00:00, 945kB/s]

text_encoder/config.json: 100%|██████████| 618/618 [00:00<?, ?B/s] 






tokenizer/tokenizer_config.json: 100%|██████████| 737/737 [00:00<?, ?B/s] 



Fetching 11 files:  18%|█▊        | 2/11 [00:01<00:06,  1.44it/s]
tokenizer/merges.txt: 100%|██████████| 525k/525k [00:00<00:00, 1.56MB/s]
tokenizer/tokenizer.json: 100%|██████████| 2.22M/2.22M [00:00<00:00, 3.70MB/s]


tokenizer/vocab.json: 100%|██████████| 862k/862k [00:00<00:00, 1.90MB/s]


























































































































































































































































model.safetensors: 100%|██████████| 2.77G/2.77G [01:35<00:00, 28.9MB/s]





























































































In [5]:
# Move models to the chosen device
teacher_model = teacher_model.to(device)
unet_model = unet_model.to(device)

Setup Loss Function and Optimizer

In [6]:
class HingeLoss(torch.nn.Module):
    def __init__(self):
        super(HingeLoss, self).__init__()

    def forward(self, output, target):
        hinge_loss = 1 - torch.mul(output, target)
        hinge_loss[hinge_loss < 0] = 0  # Equivalent to max(0, 1 - y*output)
        return torch.mean(hinge_loss)


# Hinge loss for adversarial loss and L2 loss for distillation
adversarial_loss_function = HingeLoss()
distillation_loss_function = torch.nn.MSELoss()

# Set up the optimizer for the UNet model
student_optimizer = torch.optim.Adam(unet_model.parameters(), lr=0.0001)

# Assuming you have a discriminator model defined
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0001)

#Data transformation

In [7]:
dataset = load_dataset("poloclub/diffusiondb")

# Select the split you want to use, usually 'train' for training
train_dataset = dataset['train']

print(train_dataset)

# Define transformations if needed (depending on model input requirements)
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    # Add other necessary transformations (e.g., resizing, normalization)
])

def transform_data(batch):
    return {'image': [transform(image) for image in batch['image']], 'prompt': batch['prompt']}

def custom_collate_fn(batch):
    batch_images = torch.Tensor([item['image'] for item in batch])
    batch_prompts = [item['prompt'] for item in batch]
    return {'image': batch_images, 'prompt': batch_prompts}

# Apply transformations in batches
train_dataset = train_dataset.map(transform_data, batched=True, batch_size=32)


# Use the custom collate function in DataLoader
dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)

Dataset({
    features: ['image', 'prompt', 'seed', 'step', 'cfg', 'sampler', 'width', 'height', 'user_name', 'timestamp', 'image_nsfw', 'prompt_nsfw'],
    num_rows: 1000
})


#Training

In [11]:
def show_tensor_image(pic):

    # Convert the tensor to a PIL Image
    if (isinstance(pic, torch.Tensor)):
        pil_image = transforms.ToPILImage()(pic)

    else:
        pil_image = pic

    # Display the image
    plt.imshow(pil_image)
    plt.axis('off')  # Turn off axis numbers
    plt.show()


for epoch in range(100):
    for batch in dataloader:

        for image, prompt in zip(batch['image'], batch['prompt']):

            generated_image = teacher_model(prompt, width=128, height=128).images[0]
            # Call the function to display the image
            show_tensor_image(generated_image)
            show_tensor_image(image)
            print(prompt)


Token indices sequence length is longer than the specified maximum sequence length for this model (92 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['z cushart and artem demura and alphonse mucha, masterpiece']
100%|██████████| 60/60 [00:04<00:00, 14.05it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (92 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['z cushart and artem demura and alphonse mucha, masterpiece']
  0%|          | 0/12 [00:00<?, ?it/s]


RuntimeError: pixel_unshuffle expects height to be divisible by downscale_factor, but input.size(-2)=21 is not divisible by 2